# COMP47670 Data Science with Python- Assignment 1 Notebook 1

### Irem Erkilic 24220454

## Task 1: Data Collection

In [1]:
!pip install beautifulsoup4
!pip install regex
!pip install requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.0/186.0 kB 22.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# Import necessary packages
import pandas as pd
import json, requests, urllib
from pathlib import Path
from datetime import datetime
from bs4 import BeautifulSoup
import regex
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

First, I had to figure out the URL structure - turns out the site organizes data by quarters (Q1-Q4), with each quarter having multiple pages (15-17 each). After importing BeautifulSoup and requests, I wrote a script that loops through all 62 pages and extracts data from HTML tables. The raw data needed serious cleaning since customer details were put together and date formats were inconsistent (some showing as "01/01/2024" while others were "14/January/2024"). I used regex patterns to split customer information into separate columns and standardized all dates with pandas' datetime function. Also fixed the price fields by removing currency symbols and converting to floats. Finally saved everything to a CSV file with 1,800 clean records for analysis. The hardest part was definitely handling those inconsistently formatted customer details - had to rewrite that extraction function a couple times to get it right.

In [3]:
# Base URL
base_url = "http://mlg.ucd.ie/modules/python/assignment1/retail/"

# Get the main page
response = requests.get(base_url + "index.html")
soup = BeautifulSoup(response.content, 'html.parser')

# Extract all hyperlinks
quarterly_links = []
for a_tag in soup.find_all('a'):
    href = a_tag.get('href')
    if href and 'Q' in href and 'page01' in href:  # Find first page of each quarter
        quarterly_links.append(href)

# Dictionary to hold maximum pages for each quarter
max_pages = {'Q1': 15, 'Q2': 15, 'Q3': 17, 'Q4': 15}

# Generate all page URLs
all_page_urls = []
for quarter, max_page in max_pages.items():
    for page in range(1, max_page + 1):
        page_str = f"{page:02d}"  # Format as 01, 02, etc.
        all_page_urls.append(f"{base_url}2024{quarter}-page{page_str}.html")


print(f"Found {len(all_page_urls)} pages to scrape")

# Scrape data from all pages
all_data = []

for page_url in all_page_urls:
    try:
        print(f"Scraping: {page_url}")
        page_response = requests.get(page_url)
        page_soup = BeautifulSoup(page_response.content, 'html.parser')
        
        # Find all transaction blocks
        transaction_blocks = page_soup.find_all('table')
        
        for table in transaction_blocks:
            # Create a dictionary for this transaction
            transaction = {}
            
            # Add source information
            transaction['source_url'] = page_url
            
            # Extract data from each row in the table
            rows = table.find_all('tr')
            for row in rows:
                # Get the header (field name) and value
                cells = row.find_all(['th', 'td'])
                if len(cells) == 2:  # Make sure we have a key-value pair
                    field_name = cells[0].text.strip().rstrip(':')  # Remove trailing colon if present
                    field_value = cells[1].text.strip()
                    transaction[field_name] = field_value
            
            # Add the complete transaction to our data
            if transaction:  
                all_data.append(transaction)
                
    except Exception as e:
        print(f"Error scraping {page_url}: {e}")

# Convert to DataFrame
df = pd.DataFrame(all_data)
print(f"Total records collected: {len(df)}")
print(df.head())

def extract_customer_details(details):
    customer = {'Customer ID': None, 'Location': None, 'Gender': None, 'Age Category': None}
    
    # Extract ID - look for digits after "ID:"
    id_match = regex.search(r'ID:\s*(\d+)', details)
    if id_match:
        customer['Customer ID'] = id_match.group(1)
    
    # Extract Location - look for text between "Location:" and the next keyword
    location_match = regex.search(r'Location:\s*([^G|I|A]+)', details)
    if location_match:
        customer['Location'] = location_match.group(1).strip()
    
    # Extract Gender - look for Male or Female after "Gender:"
    gender_match = regex.search(r'Gender:\s*(Male|Female)', details)
    if gender_match:
        customer['Gender'] = gender_match.group(1).strip()
    
    # Extract Age Category - look for digits-digits after "Age Category:"
    age_match = regex.search(r'Age Category:\s*(\d+-\d+)', details)
    if age_match:
        customer['Age Category'] = age_match.group(1).strip()
    
    return pd.Series(customer)

# Apply the function to create new columns
customer_details_df = df['Customer Details'].apply(extract_customer_details)
df = pd.concat([df, customer_details_df], axis=1)

# Drop the original Customer Details column
df = df.drop(columns=['Customer Details'])

# Clean price and profit columns
df['Total Price'] = df['Total Price'].str.replace('€', '').str.strip().astype(float)
df['Total Profit'] = df['Total Profit'].str.replace('€', '').str.strip().astype(float)
df['Quantity'] = df['Quantity'].astype(int)

# Standardize date formats
df['Date of Sale'] = pd.to_datetime(df['Date of Sale'], format='mixed')

# Split product categories
df[['Main Category', 'Subcategory']] = df['Product Category'].str.split(' → ', expand=True)

# Print the cleaned DataFrame to verify
print("\nCleaned data structure:")
print(df.head())

# Save to CSV
df.to_csv('sales_data.csv', index=False)


Found 62 pages to scrape
Scraping: http://mlg.ucd.ie/modules/python/assignment1/retail/2024Q1-page01.html
Scraping: http://mlg.ucd.ie/modules/python/assignment1/retail/2024Q1-page02.html
Scraping: http://mlg.ucd.ie/modules/python/assignment1/retail/2024Q1-page03.html
Scraping: http://mlg.ucd.ie/modules/python/assignment1/retail/2024Q1-page04.html
Scraping: http://mlg.ucd.ie/modules/python/assignment1/retail/2024Q1-page05.html
Scraping: http://mlg.ucd.ie/modules/python/assignment1/retail/2024Q1-page06.html
Scraping: http://mlg.ucd.ie/modules/python/assignment1/retail/2024Q1-page07.html
Scraping: http://mlg.ucd.ie/modules/python/assignment1/retail/2024Q1-page08.html
Scraping: http://mlg.ucd.ie/modules/python/assignment1/retail/2024Q1-page09.html
Scraping: http://mlg.ucd.ie/modules/python/assignment1/retail/2024Q1-page10.html
Scraping: http://mlg.ucd.ie/modules/python/assignment1/retail/2024Q1-page11.html
Scraping: http://mlg.ucd.ie/modules/python/assignment1/retail/2024Q1-page12.html
Scr

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=430c697a-6a7e-401e-9d0a-5bd226fea66f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>